# Visualization of MHD disk

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from astropy import constants as c

import volrender

Read **STEFAN HEIGLS** data. Data is currently (until 01.02.2022) available [here](https://gigamove.rwth-aachen.de/de/download/4abe80f1c550806021f85af8c57c886e).

In [ ]:
with np.load('../data/mhddisk.npz') as f:
    data = f['arr_0'][()]

Normalize

In [ ]:
vmax = data.max()
lognorm = LogNorm(vmin=vmax * 1e-4, vmax=vmax, clip=True)
datacube = lognorm(data.ravel()).reshape(data.shape).data

define a transfer function

In [ ]:
tf = volrender.TransferFunction(
    x0 = [0.1, 0.4, 0.92],
    sigma = [0.03, 0.05, 0.03],
    colors = np.array([
        [1.  , 0.5 , 0.  , 0.004],
        [0.25, 0.25, 0.75, 0.05],
        [1.  , 0.  , 0.25, 0.4 ]])
    )
tf.colors[1:,-1] /= 1.5

create the renderer

In [ ]:
ren = volrender.Renderer(datacube, interactive=False, tf=tf, N=500)

In [ ]:
v = (c.pc/c.au).cgs.value

L  = 0.004 # box length in pc
L  *= v
M  = 1.19046233 # central mass in solar masses
# position of the sink
x0 = 0.46407932
y0 = 0.55431781
z0 = 0.49871549

In [ ]:
ren.render(theta=50, phi=300, bg=1.0)

## Plotting

### Single plot with colorbar

In [ ]:
from matplotlib.colors import Normalize

In [ ]:
im = ren.image.copy()
alpha = im[:, :, -1]

vmax = alpha.max()
vmin = alpha.min()

alpha = LogNorm(vmin=vmin, vmax=vmax, clip=True)(alpha)

im[:, :, -1] = alpha
f,ax = plt.subplots()
ax.set_facecolor('none')
ax.imshow(im.transpose(1,0,2), origin='lower');

In [ ]:
f, ax = ren.plot(cb_norm=lognorm, alpha=(1 - alpha**4), invert=True)
im=ax.get_children()[0]
im.set_array(im.get_array().transpose(1,0,2)[:,:,:])
f.savefig('output/render2.pdf', transparent=True, bbox_inches='tight')

In [ ]:
f, ax = plt.subplots(dpi=200)
x = np.linspace(-L/2, L/2, data.shape[0] + 1)

data_sum = data.sum(2) * 0.004 * c.pc.to('cm').value
vmax = data_sum.max()
vmin = vmax*1e-3

cc=ax.pcolormesh(x, x, data_sum, norm=LogNorm(vmin=vmin, vmax=vmax), cmap='Blues')
ax.set_aspect('equal')
ax.set_xlabel('x [au]')
ax.set_ylabel('y [au]')

cb = plt.colorbar(cc, pad=0, ax=ax)
cb.set_label('column density [g/cm$^2$]', fontsize='small')

### Plot with histogram

In [ ]:
f, ax = ren.plot(cb_norm=lognorm, diagnostics=True)
f.subplots_adjust(wspace=0.5)

### Make a movie

In [ ]:
N = 360
theta = np.linspace(25, 25, N)
phi = np.linspace(0, 359, N)

In [ ]:
fname = 'output/movie_mhddisk.mp4'
volrender.render_helper.render_movie(datacube, theta, phi, ncpu=1, fname=fname, tf=tf, N=500, dpi=600, bg=1.0)

In [ ]:
from IPython.display import HTML
HTML(f"""
<video width="500" controls loop autoplay>
  <source src="{fname}" type="video/mp4">
</video>
""")